In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import libraries
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
#Import Dataset + Data Preprocessing
df_mitbih_train = pd.read_csv(r"/content/drive/MyDrive/Datasets/ecg_dataset/mitbih_train.csv", header=None)
df_mitbih_test = pd.read_csv(r"/content/drive/MyDrive/Datasets/ecg_dataset/mitbih_test.csv", header=None)

df_mitbih_train.shape, df_mitbih_test.shape

((87554, 188), (21892, 188))

In [ ]:
X_train = df_mitbih_train.drop(columns=[187])
y_train = df_mitbih_train[187]
X_test = df_mitbih_test.drop(columns=[187])
y_test = df_mitbih_test[187]

In [ ]:
model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, objective = 'multi:softmax', n_jobs = -1, num_class = 5, learning_rate = 0.1)
model.fit(X_train, y_train)
print(); print(model)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', ...)


In [ ]:
expected_y  = y_test
predicted_y = model.predict(X_test)

print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99     18118
         1.0       0.94      0.61      0.74       556
         2.0       0.97      0.89      0.92      1448
         3.0       0.86      0.69      0.77       162
         4.0       0.99      0.95      0.97      1608

    accuracy                           0.97     21892
   macro avg       0.95      0.83      0.88     21892
weighted avg       0.97      0.97      0.97     21892



In [ ]:
n_estimators = range(0, 16000, 1000)
param_grid = dict(n_estimators=n_estimators)

grid_cv = GridSearchCV(model, param_grid, return_train_score = True, verbose = 3)
_ = grid_cv.fit(X_test, y_test)

print(f'Best Parameters:', grid_cv.best_params_)
print(f'Best Score:', grid_cv.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END n_estimators=0;, score=(train=0.828, test=0.828) total time=   1.5s
[CV 2/5] END n_estimators=0;, score=(train=0.828, test=0.828) total time=   1.3s
[CV 3/5] END n_estimators=0;, score=(train=0.828, test=0.828) total time=   1.2s
[CV 4/5] END n_estimators=0;, score=(train=0.828, test=0.828) total time=   1.3s
[CV 5/5] END n_estimators=0;, score=(train=0.828, test=0.828) total time=   1.2s
[CV 1/5] END n_estimators=1000;, score=(train=1.000, test=0.976) total time=  19.8s
[CV 2/5] END n_estimators=1000;, score=(train=1.000, test=0.969) total time=  18.5s
[CV 3/5] END n_estimators=1000;, score=(train=1.000, test=0.973) total time=  19.3s
[CV 4/5] END n_estimators=1000;, score=(train=1.000, test=0.977) total time=  18.7s
[CV 5/5] END n_estimators=1000;, score=(train=1.000, test=0.973) total time=  19.4s
[CV 1/5] END n_estimators=2000;, score=(train=1.000, test=0.976) total time=  30.1s
[CV 2/5] END n_estimators=2000